Overview of Revit Integration with Speckle
This Jupyter Notebook outlines the integration between Revit and Speckle, providing code snippets and workflows for various functionalities.

Table of Contents
Setup and Initialization
Creating Branches
Committing Changes
Error Handling
Future Work
Setup and Initialization
This section includes code to initialize the Speckle client and connect to Revit.

In [5]:
%%capture
%pip install specklepy

# dotenv is a library that allows you to load environment variables from a .env file
%pip install python-dotenv
%reload_ext dotenv
%dotenv

In [6]:
import os
HOST_SERVER = os.getenv('HOST_SERVER')
ACCESS_TOKEN = os.getenv('ACCESS_TOKEN')

In [7]:
from specklepy.api.client import SpeckleClient

client = SpeckleClient(host=HOST_SERVER)  # or whatever your host is
client.authenticate_with_token(ACCESS_TOKEN)  # or whatever your token is

In [17]:
stream_id = "538fcacdbe"  # or whatever your stream id is

stream = client.stream.get(stream_id)

In [18]:
stream_id = "538fcacdbe"  # or whatever your stream id is
commit_id= "b8f8784a98"

commit = client.commit.get(commit_id=commit_id, stream_id=stream_id)

In [27]:
commit_payload_id = commit.referencedObject

from specklepy.transports.server import ServerTransport
from specklepy.api import operations

transport = ServerTransport(client=client, stream_id=stream_id)
commit_payload = operations.receive(obj_id=commit_payload_id, remote_transport=transport)

In [25]:
from specklepy.objects import Base
from collections.abc import Mapping, Iterable


def should_include(obj):
    """Define a logic for what objects to include in the diff"""
    return any(
        [
            hasattr(obj, "displayValue"),
            hasattr(obj, "speckle_type")
            and obj.speckle_type == "Objects.Organization.Collection",
            hasattr(obj, "displayStyle"),
        ]
    )


def flatten(obj, visited=None, include_func=should_include):
    """Flattens nested object structures based on given criteria.

    Parameters:
        obj: The object to flatten.
        visited: Set of objects that have already been visited to avoid circular references.
        include_func: Function to determine if an object should be included in the output.
    """
    if visited is None:
        visited = set()

    if obj in visited:
        return

    visited.add(obj)

    if include_func(obj):
        yield obj

    props = obj.__dict__ if hasattr(obj, "__dict__") else {}

    for prop in props:
        value = getattr(obj, prop)

        if value is None:
            continue

        if isinstance(value, Base):
            yield from flatten(value, visited, include_func)

        elif isinstance(value, Mapping):
            for dict_value in value.values():
                if isinstance(dict_value, Base):
                    yield from flatten(dict_value, visited, include_func)

        elif isinstance(value, Iterable):
            for list_value in value:
                if isinstance(list_value, Base):
                    yield from flatten(list_value, visited, include_func)

In [29]:
all_objects = list(flatten(commit_payload))

In [30]:
from specklepy.objects.other import Collection

new_commit = Collection(collectionType="flat_list", name="Processed Data", elements=[])

new_commit.elements = all_objects

In [31]:
# Send the points using a specific transport
newHash = operations.send(base=new_commit, transports=[transport])

In [65]:
from specklepy.logging.exceptions import GraphQLException


def new_branch(stream_id: str, branch_name: str) -> str:
    """Creates a new branch in a stream.

    Parameters:
        stream_id: The stream id.
        branch_name: The name of the new branch.
    """
    try:
        client.branch.create(stream_id, name=branch_name)
    finally:
        return client.branch.get(stream_id, name=branch_name)

In [66]:
branch = new_branch(stream_id=stream_id, branch_name="jssadb")

commit_id = client.commit.create(
        stream_id,
        newHash,
        branch.name,
        message="Commit",
    )